các bước code được thực hiện theo link: https://docs.sqlalchemy.org/en/latest/orm/tutorial.html


In [2]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
import sqlalchemy

# kiểm tra version

In [3]:
print('version: ', sqlalchemy.__version__)

version:  1.2.15


# kết nối database

In [4]:
engine = create_engine('sqlite:///:memory:', echo=False)

echo: là cờ để cài đặt/kích hoạt logging cho SQLAlchemy, nó dựa trên logging của python nếu bật nó thì sẽ nhìn thấy tất cả các câu lệnh SQL được tạo để tạo url kết nối đến các loại database khác như mysql, posgresql.. tham khảo link: https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls

# khai báo một mapping (hay 1 class mô tả 1 bảng)
khi sử dụng ORM, tiến trình bắt đầu bằng cách mô tả các bảng dử liệu, sau đó là định nghĩa các class được ánh xạ đến các bảng trên. trong SQLAlchemy 2 việc trên được thực hiện cùng nhau sử dụng Declarative tạo ra một lớp cơ sở (Base). Ứng dụng thường chỉ có duy nhất một Base class

In [5]:
Base = declarative_base()

bây giờ chúng ta sẽ định nghĩa các class mô tả bảng dữ liệu kế thừa từ Base class
tham số 'bind' gắn Base vào kết nối đến cơ sở dữ liệu được khai báo ở trên

In [7]:
class User(Base):
    __tablename__ = 'users'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        # hàm này trả về định dạng dễ đọc của đối tượng User
        return "<Users(name='{}', fullname='{}', password={})>".format(
            self.name, self.fullname, self.password
        )

print('User.__table__: ', User.__table__)

User.__table__:  users


/home/xuananh/data/repo/python-note/.venv/python3/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


Ngoài ra có thể tạo các hàm trợ giúp sử dụng minxin class:
https://docs.sqlalchemy.org/en/latest/orm/extensions/declarative/mixins.html#declarative-mixins


# tạo Schema

In [9]:
Base.metadata.create_all(engine)

# tạo một Instance của Mapped Class

In [11]:
ed_user = User(name='ed',
               fullname='Ed Jones',
               password='edspassword')
print(ed_user.id)
print(ed_user.name)
print(ed_user.password)

None
ed
edspassword


lưu ý là khai báo trên vẫn chưa được commit lên database server, vì vậy id sẽ là None


# tạo Session
chúng ta đã sẵn sàng nói chuyện với database, điều khiển quá trình này là Session khi cài đặt ứng dùng lần đầu, cùng chỗ gọi hàm create_engine(), chúng ta định nghĩa một class Session phục vụ như một nhà máy (factory) tạo mới các đối tượng Session

In [12]:
Session = sessionmaker(bind=engine)

trong trường hợp chưa không muốn gắn engine luôn thì khai báo như sau:

In [13]:
Session1 = sessionmaker()

khi nào nào tạo engine và kết nối đến nó sử dụng ham **configure()**


In [14]:
Session1.configure(bind=engine)

xong, bây giờ, khi nào cần nói chuyện với database thì tạo 1 session

In [15]:
session = Session()

session trên liên kết với engine nhưng chưa mở 1 kết nối nào, khi lần đầu kết sử dụng, nó sẽ lấy một kết nối từ một nhóm kết nối (pool connections) giữ bởi engine, và giữ kết nối này đến khi commit thay đổi hoặc đóng session object

# thêm và cập nhật đối tượng/bảng

In [17]:
ed_user = User(name='ed',
               fullname='Ed Jones',
               password='edspassword')
session.add(ed_user)

lúc này đối tượng đang chờ, chưa có SQL nào được phát ra và đối tượng chưa được mô tả bởi một hàng nào trong database. Session sẽ phát ra SQL để duy trì Ed Jones ngay khi cần, sử dụng một tiến trình gọi là phun (flush). Nếu chúng ta truy vấn database để lấy Ed Jones, tất cả những thông tin đang chờ sẽ được phun ra và truy vấn được phát ra ngay lập tức sau đó. 
Ví dụ, bên dưới ta tạo 1 đối tượng truy vấn (Query) lạp vào đối tượng User, lọc theo name = ed, và chỉ lấy kết quả đầu tiên trong danh sách đầy đủ các của các hàng. Một đối tượng User
được trả về tương đương với đối tượng ta đã tạo ở trên

In [18]:
our_user = session.query(User).filter_by(name='ed').first()
print('----- our_user = ', our_user)

----- our_user =  <Users(name='ed', fullname='Ed Jones', password=edspassword)>


trong thực tế, Session đã nhận ra rằng hàng trả về tương tự một hàng được thể hiện trong bản đồ đối tượng bên trong của nó (its internal map of object), vì vậy chúng ta thực sự lấy lại đúng đối tượng mà chúng ta đã thêm vào ở trên:

In [19]:
print('----- ed_user is our_user = ', ed_user is our_user)

----- ed_user is our_user =  False


Khái niệm ORM được biết như là một bản đồ nhận dạng và đảm bảo tất cả các hoạt động trên một hàng cụ thể trong **Session** hoạt động trên cùng một bộ dữ liệu. Khi một đối tượng với một **primary key** được biểu diễn trong **Session**, tất cả các truy vấn SQL trong **Session** đó luôn trả về cùng một đối tượng Python cho **primary key** đó, sẽ có lỗi nếu cố gắn tạo một đối tượng thứ 2 cùng **primary key** với một đối tượng đã tồn tại sẵn.
Chúng ta có thể thêm nhiều đối tượng **User** trong một lệnh **add_all()**

In [20]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

ta đã tạo một password không quá bảo mật cho người dùng Ed, hãy thay đổi nó:

In [21]:
ed_user.password = 'f8s7ccs'

Session đang để ý, nó biết Ed Jones đã bị thay đổi

In [22]:
print('----- session.dirty: ', session.dirty)

----- session.dirty:  IdentitySet([<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>])


và 3 đối tượng mới đang chờ:

In [23]:
print('----- session.new: ', session.new)

----- session.new:  IdentitySet([<Users(name='wendy', fullname='Wendy Williams', password=foobar)>, <Users(name='mary', fullname='Mary Contrary', password=xxg527)>, <Users(name='fred', fullname='Fred Flinstone', password=blah)>])


bây giờ, ta yêu cầu **Session** phát tất cả các thay đổi đến database server và commit giao dịch được thực hiện từ đầu đến giờ. Để làm việc này dùng hàm **commit()**. **Session** phát ra biểu thức **UPDATE** để thay đổi password, và **INSERT** để thêm 3 đối tượng mới.

In [24]:
session.commit()

connection resource được tham chiếu bởi session giờ trả lại cho connection pool, và sẽ lấy lại mỗi khi cần, bây giờ nếu xem id của Ed, nó sẽ không còn None nữa, mà có giá trị là:

In [25]:
print('----- ed_user.id: ', ed_user.id)

----- ed_user.id:  2


sau khi **Session** chèn hàng mới vào database, tất cả các định danh mới được tạo ra và bộ tạo database mặc định có sẵn trên các đối tượng, ngay lập tức hoặc qua lần truy cập đầu tiên. Trong trường hợp này toàn bộ các hàng được tải lại vì một phiên giao dịch mới bắt đầu sau khi chúng ta gọi **commit()**.
SQLAlchemy mặc định làm mới dữ liệu từ phiên giao dịch trước khi lần đầu truy cập một phiên giao dịch mới, vì vậy trạng thái gần nhất là có sẵn. Mức độ tải lại có thể được cấu hình như mô tả trong link: https://docs.sqlalchemy.org/en/latest/orm/session.html

# khôi phục/quay lại
Khi **Session** làm việc bên trong một phiên giao dịch, ta có thể khôi phục thay đổi đã làm. Hãy tạo 2 thay 
đổi và khổi phục, tên của ed_users là Edwardo:

In [26]:
ed_user.name = 'Edwardo'

và ta sẽ thêm một user lỗi khác, fake_user:

In [28]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

truy vấn session, ta có thể thấy rằng nó đã được phun (flushed) vào phiên giao dịch hiện tại

In [29]:
print('----- ', session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all())

-----  [<Users(name='Edwardo', fullname='Ed Jones', password=f8s7ccs)>, <Users(name='fakeuser', fullname='Invalid', password=12345)>, <Users(name='fakeuser', fullname='Invalid', password=12345)>]


chạy khôi phục, ta có thể thấy rằng name của ed_used trở lại ed, và fake_user bị đá ra khỏi session

In [30]:
session.rollback()
print('----- ed_user.name: ', ed_user.name)
print('----- fake_user in session: ', fake_user in session)

----- ed_user.name:  ed
----- fake_user in session:  False


chạy lệnh SELECT để minh họa những thay đổi với database

In [31]:
print('----- ', session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all())

-----  [<Users(name='ed', fullname='Ed Jones', password=edspassword)>, <Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>]


# truy vấn (Quering)
Một đối tượng Query được tạo bởi hàm **query()** trong **Session**. Hàm này lấy một số đối số, có thể là kết hợp của các class và bộ mô tả công cụ lớp (class-instrumented). 

In [32]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

ed Ed Jones
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


**Query** cũng chấp nhận bộ mô tả ORM-instrumented như đối số, bất cứ khi nào mà các thực thể lớp hoặc thực thể dựa trên các cột được biểu diễn như các đối số đầu vào của hàm **query()**, kết quả trả vể được biểu diễn như các bộ (tuples):

In [33]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

ed Ed Jones
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


Tên của tuple trả về tương tự tên của thuộc tính đối với một thuộc tính, và tương tự tên class đối với 1 class

In [34]:
for row in session.query(User, User.name).all():
    print(row.User, '-----', row.name)

<Users(name='ed', fullname='Ed Jones', password=edspassword)> ----- ed
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)> ----- ed
<Users(name='wendy', fullname='Wendy Williams', password=foobar)> ----- wendy
<Users(name='mary', fullname='Mary Contrary', password=xxg527)> ----- mary
<Users(name='fred', fullname='Fred Flinstone', password=blah)> ----- fred


có thể điều khiển tên của các biểu diễn cột sử dụng **label()**, có sẵn ở đối tượng dẫn suất **ColumnElement**, cũng như bất cứ thuộc tính class nào được ánh xạ đến cột đó (như **User.name**):

In [35]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

ed
ed
wendy
mary
fred


khi có nhiều thực thể được mô tả trong **query()** có thể dùng **aliased()**:

In [36]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>
<Users(name='mary', fullname='Mary Contrary', password=xxg527)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


các hoạt động cơ bản với **Query** bào gồm **LIMIT** và **OFFSET**, thuận tiện nhất là sử dụng các lát cắt mảng Python và thường kết hợp với **ORDER BY**

In [37]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>


và lọc kết quả, dùng **filter_by()**, truyền vào một keyword:

In [38]:
for name in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

('ed',)
('ed',)


hoặc **filter()**, hàm này sử dụng cấu trúc ngôn ngữ biểu thức SQL linh hoạt hơn, điều này cho phép sử dụng các toán tử Python với các thuộc tính class-level trong mapped class:

In [39]:
for name in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

('ed',)
('ed',)


đối tượng **Query** sinh ra một cách đầy đủ, nghía là hầu hết các cuộc gọi phương thức trả về  một đối tương **Query** mới, sau đó có thể thêm các tiêu chi khác nữa. Ví dụ, để truy vấn tên 'ed' và tên đầy đủ 'Ed Jones', có thể dùng **filter()** 2 lần, nó sẽ join 2 tiêu chí sử dụng **AND**:

In [40]:
for user in session.query(User).\
            filter(User.name=='ed').\
            filter(User.fullname=='Ed Jones'):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>


## Các toán tử filter chung
danh sách các toán tử hay sử dụng trong **filter()**
    - equals:

In [41]:
for user in session.query(User).filter(User.name == 'ed'):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>


    - not equals:

In [42]:
for user in session.query(User).filter(User.name != 'ed'):
    print(user)

<Users(name='wendy', fullname='Wendy Williams', password=foobar)>
<Users(name='mary', fullname='Mary Contrary', password=xxg527)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


    - LIKE (không phân biệt chữ hoa chữ thường):

In [43]:
for user in session.query(User).filter(User.name.like('%ed%')):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


    - ILIKE (phân biệt chữ hoa chữ thường):

In [44]:
for user in session.query(User).filter(User.name.ilike('%ed%')):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


    - IN:

In [45]:
for user in session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>


    - NOT IN:

In [46]:
for user in session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

<Users(name='mary', fullname='Mary Contrary', password=xxg527)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


    - IS NULL:

In [47]:
for user in session.query(User).filter(User.name == None):
    print(user)

for user in session.query(User).filter(User.name.is_(None)):
    print(user)    

    - IS NOT NULL:

In [48]:
for user in session.query(User).filter(User.name != None):
    print(user)
    
for user in session.query(User).filter(User.name.isnot(None)):
    print(user)    

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>
<Users(name='mary', fullname='Mary Contrary', password=xxg527)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>
<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>
<Users(name='mary', fullname='Mary Contrary', password=xxg527)>
<Users(name='fred', fullname='Fred Flinstone', password=blah)>


    - AND:

In [49]:
# sử dụng and_()
from sqlalchemy import and_
for user in session.query(User).filter(and_(User.name == 'ed', User.fullname == 'Ed Jones')):
    print(user)
    
# hoặc gửi nhiều biểu thức đến filter()
for user in session.query(User).filter(User.name == 'ed', User.fullname == 'Ed Jones'):
    print(user)
    
# hoặc xâu chuỗi nhiều cuộc gọi filter()/filter_by()
for user in session.query(User).filter(User.name == 'ed').filter(User.fullname == 'Ed Jones'):
    print(user)
    

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>


    - OR:

In [52]:
from sqlalchemy import or_
for user in session.query(User).filter(or_(User.name == 'ed', User.name == 'wendy')):
    print(user)

<Users(name='ed', fullname='Ed Jones', password=edspassword)>
<Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>
<Users(name='wendy', fullname='Wendy Williams', password=foobar)>


    - MATCH:
match() sử dụng hàm dữ liệu đặc biệt MATCH hoặc CONTAINS, hành vi này khác nhau theo database và không có sẵn trong một vài database, ví dụ như SQLite, ví thế hàm bên dưới sẽ báo lỗi trong ví dụ này

In [54]:
for user in session.query(User).filter(User.name.match('wendy')):
    print(user)

OperationalError: (sqlite3.OperationalError) unable to use function MATCH in the requested context [SQL: 'SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password \nFROM users \nWHERE users.name MATCH ?'] [parameters: ('wendy',)] (Background on this error at: http://sqlalche.me/e/e3q8)

## trả về danh sách hoặc vô hướng
một số phương thức trong Query ngay lập tức phát ra SQL và trả về một giá trị chứa kết quả dữ liệu được tải
    - all() trả về một danh sách:

In [56]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
print(query.all())

[<Users(name='ed', fullname='Ed Jones', password=edspassword)>, <Users(name='ed', fullname='Ed Jones', password=f8s7ccs)>, <Users(name='fred', fullname='Fred Flinstone', password=blah)>]


    - first() trả về kết quả đầu tiên dưới dạng vô hướng:

In [59]:
print(query.first())

<Users(name='ed', fullname='Ed Jones', password=edspassword)>


    - one() lấy đầy đủ tất cả các hàng, nếu không chính xác 1 hàng tìm thấy hoặc hoặc tập hợp của các hàng được mô tả trong kết quả, đưa ra lỗi , Ví dụ với nhiều hàng tìm thấy:

In [61]:
print(query.one())

MultipleResultsFound: Multiple rows were found for one()

ví dụ không tìm thấy hàng nào:

In [63]:
print(query.filter(User.id==99).one())

NoResultFound: No row was found for one()

**one()** toẹt vời cho hệ thống muốn xử lý 'no items found' khác với 'multiple items found' , ví dụ RESTful có thể muốn phát ra lỗi '404 not found' khi không tìm thấy kết quả nào, nhưng phát ra một lỗi ứng dụng khi có nhiều kết quả tìm thấy.

    - one_or_none() giống như one(), mong đợi không tìm thấy kết quả nào, nó không phát ra lỗi, chỉ tra về None. Giống one(), tuy nhiên nó phát ra lỗi khi tìm thấy nhiều kết quả.
    
    - scalar() gọi phương thức one(), và khi thành công trả về cột đầu tiên của hàng:

In [66]:
query = session.query(User.id).filter(User.name=='ed').order_by(User.id)
print(query.scalar())

MultipleResultsFound: Multiple rows were found for one()

## Sử dụng nguyên văn câu lệnh SQL
Các chuỗi ký tự có thể được sử dụng mềm dẻo với **Query**, bằng cách sử dụng **text()**, nó được chấp nhận phần lớn các phương thức mà ta có thể sử dụng được với **Query**. Ví dụ, **filter()** và **order_by()**:

In [69]:
from sqlalchemy import text
for user in session.query(User).\
            filter(text("id<224")).\
            order_by(text("id")).all():
    print(user.name)

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper|Address|addresses'. Original exception was: Mapper 'Mapper|User|users' has no property 'address'

Tham số truyền vào câu lệnh SQL sử dụng dấu hai chấm (:). Để chỉ định giá trị cho tham số, dùng hàm **params()**

In [ ]:
query = session.query(User).filter(text("id<:value and name=:name")).\
                params(value=224, name='fred').\
                order_by(User.id).one()
print(query)

để sử dụng một hoàn toàn bằng biểu thức SQL, truyền một cấu trúc **text()** mô tả một biểu thức hoàn chỉnh vào **from_statement()**. Không có các chỉ định bổ xung, các cột trong chuỗi SQL được khớp với các cột trong model dựa vào tên, chẳng hạn bên dưới ta chỉ dùng dấu hoa thị để  mô tả tải tất cả các cột:

In [ ]:
query = session.query(User).from_statement(
                            text("SELECT * FROM users WHERE name=:name")).\
                            params(name='ed').all()
print(query)


tiếp...

# Xây dựng một Relationship
Hãy xem xét làm thế nào một bảng thứ 2, liên quan đến User, có thể mapped và queried. Người dùng trong hệ thống có thể lưu bao nhiêu email liên kết với username của họ cũng được. Nó ám chỉ liên kết một-nhiều từ bảng users đến bảng mới chứa email, có thể gọi là bảng address:

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    
    user = relationship("User", back_populates='address')
    
    def __repr__(self):
        return "<Address(email_address={})".format(self.email_address)
    
User.addresses = relationship("Address", order_by=Address.id, back_populates='user')

class trên giới thiệu bộ tạo **ForeignKey**, là một chỉ dẫn áp dụng cho cột, nó chỉ ra rằng giá trị trong cột này bị ràng buộc với giá trị mô tả trong tên cột từ xa. Đây là tính năng cốt lỗi của cơ sở dữ liệu quan hệ, nó như một loại keo dán biến đổi một tập hợp không kết nối của các bảng thành các mối quan hệ chồng chéo phong phú. 
- **ForeignKey** bên trên diễn tả rằng giá trị trong cột **address.user_id** phải được ràng buộc đến các giá trị trong cột **user.id**, tức là khóa chính của nó.
- chỉ thị **relationship()** nói với ORM rằng class **Address** phải được liên kết với class **User**, sử dụng thuộc tính **Address.user**
- **relationship()** sử dụng mối quan hệ khóa ngoài giữa 2 bảng đẻ xác định bản chất của liên kết này, nó xác định rằng **Address.user** sẽ là nhiều-một
- một chỉ thị **relationship()** nữa được trên **User** class dưới  thuộc tính **User.addresses** 
- cả 2 chỉ thị **relationship()**, tham số **back_populates** được gán để tham chiếu đến tên thuộc tính bổ xung, bằng cách làm vậy, mỗi **relationship()** có thế tạo ra quyết định thông minh về cùng mối quan hệ ở phía ngược lại. Nghĩa là, một bên **Address.user** tham chiếu đến đối tượng **User**, bên khác, **User,addresses** tham chiếu đến một danh sách các đối tượng **Address**

tiếp...

# Làm việc với các Related Objects
Bây giờ, khi ta tạo một **User**, một tập hợp **address** cũng sẽ hiện diện. Có nhiều loại tập hợp khác nhau như list, set, dictionary xem [ở đây](https://docs.sqlalchemy.org/en/latest/orm/collections.html#custom-collections), nhưng mặc định trong Python là list

In [ ]:
jack = User(name = 'jack', fullname='Jack Bean', password='asdfdasf')
print(jack.addresses)